# Data Governance & Standardization Framework
## Unity Catalog Implementation Guide

This notebook establishes comprehensive data governance and standardization practices aligned with Databricks best practices using Unity Catalog.

## Framework Components
* **Catalog Structure** - Three-level namespace organization (catalog.schema.table)
* **Access Control** - Role-based permissions and privilege management
* **Schema Governance** - Enforced data types, constraints, and metadata standards
* **Quality Gates** - Automated validation checkpoints for data quality assurance
* **Audit & Compliance** - Lineage tracking and access monitoring
* **Standards Enforcement** - Automated checks for priority datasets

## 1. Unity Catalog Architecture & Setup

### Three-Level Namespace Hierarchy

```
Catalog (Top Level)
  ├── Schema (Database)
  │     ├── Tables
  │     ├── Views
  │     ├── Functions
  │     └── Volumes (for files)
  └── Schema
        └── ...
```

### Recommended Catalog Structure

**By Environment:**
* `dev_catalog` - Development and experimentation
* `staging_catalog` - Pre-production testing
* `prod_catalog` - Production data assets

**By Domain (Alternative):**
* `clinical_catalog` - Clinical and patient data
* `operational_catalog` - Operational and administrative data
* `analytics_catalog` - Analytics and reporting datasets

### Schema Organization (Medallion Architecture)
* `bronze` - Raw, unprocessed data from source systems
* `silver` - Cleaned, validated, and conformed data
* `gold` - Business-level aggregates and analytics-ready datasets
* `sandbox` - Experimental and ad-hoc analysis workspace

#### **How to Create environment-specific catalogs**

```sql
-- Create environment-specific catalogs
CREATE CATALOG IF NOT EXISTS dev_catalog
  COMMENT 'Development environment for data engineering and testing';

CREATE CATALOG IF NOT EXISTS staging_catalog
  COMMENT 'Staging environment for pre-production validation';

CREATE CATALOG IF NOT EXISTS prod_catalog
  COMMENT 'Production environment for business-critical data assets';

-- Show all catalogs
SHOW CATALOGS;
````

#### **How to create medallion architecture schemas in production catalog**

```sql
-- Create medallion architecture schemas in production catalog
USE CATALOG prod_catalog;

CREATE SCHEMA IF NOT EXISTS bronze
  COMMENT 'Raw data layer - unprocessed data from source systems'
  WITH DBPROPERTIES (
    'layer' = 'bronze',
    'data_classification' = 'raw',
    'owner_team' = 'data_engineering'
  );

CREATE SCHEMA IF NOT EXISTS silver
  COMMENT 'Cleaned data layer - validated and conformed data'
  WITH DBPROPERTIES (
    'layer' = 'silver',
    'data_classification' = 'cleaned',
    'owner_team' = 'data_engineering'
  );

CREATE SCHEMA IF NOT EXISTS gold
  COMMENT 'Business data layer - aggregated and analytics-ready'
  WITH DBPROPERTIES (
    'layer' = 'gold',
    'data_classification' = 'curated',
    'owner_team' = 'analytics'
  );

CREATE SCHEMA IF NOT EXISTS sandbox
  COMMENT 'Experimental workspace for ad-hoc analysis'
  WITH DBPROPERTIES (
    'layer' = 'sandbox',
    'data_classification' = 'experimental',
    'owner_team' = 'all_users'
  );

-- Show all schemas
SHOW SCHEMAS IN prod_catalog;
```

## 2. Access Control & Permissions Management

### Unity Catalog Privilege Model

**Securable Objects:**
* Catalog
* Schema
* Table/View
* Volume
* Function

**Key Privileges:**

| Privilege | Description | Use Case |
|-----------|-------------|----------|
| `USE CATALOG` | Access catalog | Required to see catalog contents |
| `USE SCHEMA` | Access schema | Required to see schema contents |
| `SELECT` | Read data | Analysts, reporting tools |
| `MODIFY` | Insert/Update/Delete | ETL processes |
| `CREATE TABLE` | Create tables | Data engineers |
| `ALL PRIVILEGES` | Full control | Catalog owners |

### Role-Based Access Control (RBAC) Strategy

**Data Engineering Team:**
* Full access to `dev_catalog`
* CREATE, MODIFY, SELECT on `prod_catalog.bronze` and `prod_catalog.silver`
* SELECT on `prod_catalog.gold`

**Analytics Team:**
* SELECT on all `prod_catalog` schemas
* CREATE, MODIFY on `prod_catalog.sandbox`

**Business Users:**
* SELECT on `prod_catalog.gold` only
* No access to bronze/silver layers

#### **How to Grant permissions**

```sql
-- Grant permissions to Data Engineering group
GRANT USE CATALOG ON CATALOG prod_catalog TO `data_engineering_team`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.bronze TO `data_engineering_team`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.silver TO `data_engineering_team`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.gold TO `data_engineering_team`;

GRANT CREATE TABLE ON SCHEMA prod_catalog.bronze TO `data_engineering_team`;
GRANT CREATE TABLE ON SCHEMA prod_catalog.silver TO `data_engineering_team`;
GRANT SELECT, MODIFY ON SCHEMA prod_catalog.bronze TO `data_engineering_team`;
GRANT SELECT, MODIFY ON SCHEMA prod_catalog.silver TO `data_engineering_team`;
GRANT SELECT ON SCHEMA prod_catalog.gold TO `data_engineering_team`;

-- Grant permissions to Analytics group
GRANT USE CATALOG ON CATALOG prod_catalog TO `analytics_team`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.gold TO `analytics_team`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.sandbox TO `analytics_team`;
GRANT SELECT ON SCHEMA prod_catalog.gold TO `analytics_team`;
GRANT CREATE TABLE, SELECT, MODIFY ON SCHEMA prod_catalog.sandbox TO `analytics_team`;

-- Grant permissions to Business Users group
GRANT USE CATALOG ON CATALOG prod_catalog TO `business_users`;
GRANT USE SCHEMA ON SCHEMA prod_catalog.gold TO `business_users`;
GRANT SELECT ON SCHEMA prod_catalog.gold TO `business_users`;

-- Show grants on catalog
SHOW GRANTS ON CATALOG prod_catalog;
```

In [0]:
# Permission management utility class

class PermissionManager:
    """Manage Unity Catalog permissions programmatically"""
    
    def __init__(self, catalog: str):
        self.catalog = catalog
    
    def grant_read_access(self, schema: str, principal: str):
        """Grant read-only access to a schema"""
        grants = [
            f"GRANT USE CATALOG ON CATALOG {self.catalog} TO `{principal}`",
            f"GRANT USE SCHEMA ON SCHEMA {self.catalog}.{schema} TO `{principal}`",
            f"GRANT SELECT ON SCHEMA {self.catalog}.{schema} TO `{principal}`"
        ]
        
        for grant in grants:
            print(f"Executing: {grant}")
            try:
                spark.sql(grant)
                print("  ✓ Success")
            except Exception as e:
                print(f"  ✗ Error: {str(e)}")
    
    def grant_write_access(self, schema: str, principal: str):
        """Grant write access to a schema"""
        grants = [
            f"GRANT USE CATALOG ON CATALOG {self.catalog} TO `{principal}`",
            f"GRANT USE SCHEMA ON SCHEMA {self.catalog}.{schema} TO `{principal}`",
            f"GRANT CREATE TABLE ON SCHEMA {self.catalog}.{schema} TO `{principal}`",
            f"GRANT SELECT, MODIFY ON SCHEMA {self.catalog}.{schema} TO `{principal}`"
        ]
        
        for grant in grants:
            print(f"Executing: {grant}")
            try:
                spark.sql(grant)
                print("  ✓ Success")
            except Exception as e:
                print(f"  ✗ Error: {str(e)}")
    
    def revoke_access(self, schema: str, principal: str):
        """Revoke all access from a schema"""
        revokes = [
            f"REVOKE ALL PRIVILEGES ON SCHEMA {self.catalog}.{schema} FROM `{principal}`"
        ]
        
        for revoke in revokes:
            print(f"Executing: {revoke}")
            try:
                spark.sql(revoke)
                print("  ✓ Success")
            except Exception as e:
                print(f"  ✗ Error: {str(e)}")
    
    def audit_permissions(self, schema: str = None):
        """Audit current permissions"""
        if schema:
            query = f"SHOW GRANTS ON SCHEMA {self.catalog}.{schema}"
        else:
            query = f"SHOW GRANTS ON CATALOG {self.catalog}"
        
        print(f"\nPermission Audit: {query}")
        print("=" * 80)
        
        try:
            df = spark.sql(query)
            display(df)
            return df
        except Exception as e:
            print(f"Error: {str(e)}")
            return None

# Example usage
print("Permission Manager initialized")
print("Usage: pm = PermissionManager('prod_catalog')")
print("       pm.grant_read_access('gold', 'user@example.com')")

## 3. Schema Governance & Standardization

### Table Naming Conventions

**Format:** `{domain}_{entity}_{type}`

**Examples:**
* `patient_demographics_dim` - Dimension table for patient demographics
* `clinical_encounters_fact` - Fact table for clinical encounters
* `lab_results_raw` - Raw lab results data
* `pharmacy_orders_agg` - Aggregated pharmacy orders

### Required Table Properties

All production tables must include:

In [ ]:
# Metadata template for datasets
metadata_template = {
    'owner': 'team_name',
    'domain': 'clinical|operational|analytics',
    'pii_flag': 'true|false',
    'retention_days': '365|730|2555',  # 1yr, 2yr, 7yr
    'data_classification': 'public|internal|confidential|restricted',
    'quality_tier': 'bronze|silver|gold',
    'update_frequency': 'realtime|hourly|daily|weekly|monthly',
    'source_system': 'system_name',
    'business_owner': 'owner_email'
}

### Column Naming Standards

* Use `snake_case` for all column names
* Prefix foreign keys with table name: `patient_id`, `encounter_id`
* Use standard suffixes:
  * `_id` for identifiers
  * `_date` for dates
  * `_timestamp` for timestamps
  * `_flag` for boolean indicators
  * `_code` for coded values
  * `_desc` for descriptions

### Data Type Standards

| Data Category | Standard Type | Example |
|---------------|---------------|----------|
| Identifiers | `BIGINT` or `STRING` | patient_id BIGINT |
| Dates | `DATE` | admission_date DATE |
| Timestamps | `TIMESTAMP` | created_timestamp TIMESTAMP |
| Currency | `DECIMAL(18,2)` | amount DECIMAL(18,2) |
| Percentages | `DECIMAL(5,2)` | rate DECIMAL(5,2) |
| Flags | `BOOLEAN` | is_active BOOLEAN |
| Codes | `STRING` | diagnosis_code STRING |

In [0]:
# Standardized table creation with governance metadata

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, DecimalType, BooleanType
from datetime import datetime

class GovernedTableBuilder:
    """Build tables with enforced governance standards"""
    
    def __init__(self, catalog: str, schema: str):
        self.catalog = catalog
        self.schema = schema
        self.table_properties = {}
        self.required_properties = [
            'owner', 'domain', 'pii_flag', 'retention_days',
            'data_classification', 'quality_tier', 'update_frequency',
            'source_system', 'business_owner'
        ]
    
    def set_properties(self, **kwargs):
        """Set table properties with validation"""
        self.table_properties.update(kwargs)
        return self
    
    def validate_properties(self) -> bool:
        """Validate required properties are present"""
        missing = [prop for prop in self.required_properties if prop not in self.table_properties]
        
        if missing:
            print(f"✗ Missing required properties: {', '.join(missing)}")
            return False
        
        print("✓ All required properties present")
        return True
    
    def create_table(self, table_name: str, schema: StructType, df=None, mode: str = "overwrite"):
        """Create table with governance metadata"""
        
        if not self.validate_properties():
            raise ValueError("Table properties validation failed")
        
        full_table_name = f"{self.catalog}.{self.schema}.{table_name}"
        
        # Add system properties
        self.table_properties['created_by'] = spark.sql("SELECT current_user()").collect()[0][0]
        self.table_properties['created_date'] = datetime.now().strftime("%Y-%m-%d")
        
        print(f"\nCreating governed table: {full_table_name}")
        print("=" * 80)
        print("\nTable Properties:")
        for key, value in self.table_properties.items():
            print(f"  {key}: {value}")
        
        if df is not None:
            # Create table from DataFrame
            writer = df.write.format("delta").mode(mode)
            
            # Add table properties
            for key, value in self.table_properties.items():
                writer = writer.option(f"delta.property.{key}", str(value))
            
            writer.saveAsTable(full_table_name)
            print(f"\n✓ Table created successfully with {df.count()} rows")
        else:
            # Create empty table with schema
            empty_df = spark.createDataFrame([], schema)
            writer = empty_df.write.format("delta").mode(mode)
            
            for key, value in self.table_properties.items():
                writer = writer.option(f"delta.property.{key}", str(value))
            
            writer.saveAsTable(full_table_name)
            print(f"\n✓ Empty table created successfully")
        
        return full_table_name
    
    def add_table_comment(self, table_name: str, comment: str):
        """Add descriptive comment to table"""
        full_table_name = f"{self.catalog}.{self.schema}.{table_name}"
        spark.sql(f"COMMENT ON TABLE {full_table_name} IS '{comment}'")
        print(f"✓ Comment added to {full_table_name}")
    
    def add_column_comments(self, table_name: str, column_comments: dict):
        """Add comments to columns"""
        full_table_name = f"{self.catalog}.{self.schema}.{table_name}"
        
        for column, comment in column_comments.items():
            spark.sql(f"ALTER TABLE {full_table_name} ALTER COLUMN {column} COMMENT '{comment}'")
            print(f"✓ Comment added to column: {column}")

# Example usage
print("Governed Table Builder initialized")
print("\nExample usage:")
print("""builder = GovernedTableBuilder('prod_catalog', 'silver')
builder.set_properties(
    owner='data_engineering_team',
    domain='clinical',
    pii_flag='true',
    retention_days='2555',
    data_classification='confidential',
    quality_tier='silver',
    update_frequency='daily',
    source_system='emr_system',
    business_owner='clinical.lead@example.com'
)""")

In [0]:
# Example: Create a governed patient demographics table

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, BooleanType, TimestampType
from datetime import datetime

# Define schema with standard naming conventions
patient_schema = StructType([
    StructField("patient_id", IntegerType(), False),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("date_of_birth", DateType(), True),
    StructField("gender_code", StringType(), True),
    StructField("is_active", BooleanType(), True),
    StructField("created_timestamp", TimestampType(), False),
    StructField("updated_timestamp", TimestampType(), False)
])

# Create sample data
sample_data = [
    (1, "John", "Doe", datetime(1980, 5, 15).date(), "M", True, datetime.now(), datetime.now()),
    (2, "Jane", "Smith", datetime(1992, 8, 22).date(), "F", True, datetime.now(), datetime.now()),
    (3, "Robert", "Johnson", datetime(1975, 3, 10).date(), "M", True, datetime.now(), datetime.now())
]

sample_df = spark.createDataFrame(sample_data, patient_schema)

print("Sample patient demographics data:")
display(sample_df)

print("\n✓ Sample data created with standardized schema")
print(f"  - Columns follow snake_case naming")
print(f"  - IDs use _id suffix")
print(f"  - Dates use _date suffix")
print(f"  - Timestamps use _timestamp suffix")
print(f"  - Flags use is_ prefix and BOOLEAN type")
print(f"  - Codes use _code suffix")

## 4. Quality Gates & Data Validation

### Quality Gate Levels

**Bronze Layer (Raw Data):**
* Schema validation (expected columns present)
* Data type validation
* Record count thresholds
* Duplicate detection

**Silver Layer (Cleaned Data):**
* All Bronze checks
* Null value constraints
* Referential integrity
* Business rule validation
* Data freshness checks

**Gold Layer (Curated Data):**
* All Silver checks
* Aggregation accuracy
* Metric consistency
* Historical trend validation

### Quality Dimensions

| Dimension | Description | Example Check |
|-----------|-------------|---------------|
| **Completeness** | Required fields populated | NULL count = 0 for NOT NULL columns |
| **Accuracy** | Values within valid ranges | age BETWEEN 0 AND 120 |
| **Consistency** | Data conforms to standards | date formats, code values |
| **Timeliness** | Data is current | max(updated_date) within SLA |
| **Uniqueness** | No duplicate records | DISTINCT count = total count |
| **Validity** | Values match constraints | foreign keys exist |

In [0]:
# Comprehensive quality gate framework

from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from datetime import datetime, timedelta
import json

class QualityGate:
    """Data quality validation framework"""
    
    def __init__(self, df: DataFrame, table_name: str, layer: str):
        self.df = df
        self.table_name = table_name
        self.layer = layer
        self.checks = []
        self.results = []
        self.passed = True
    
    def check_schema(self, expected_columns: list) -> 'QualityGate':
        """Validate expected columns are present"""
        actual_columns = set(self.df.columns)
        expected_columns_set = set(expected_columns)
        
        missing = expected_columns_set - actual_columns
        extra = actual_columns - expected_columns_set
        
        passed = len(missing) == 0
        self.passed = self.passed and passed
        
        self.results.append({
            'check': 'Schema Validation',
            'passed': passed,
            'details': f"Missing: {list(missing) if missing else 'None'}, Extra: {list(extra) if extra else 'None'}",
            'severity': 'CRITICAL'
        })
        return self
    
    def check_not_empty(self, min_rows: int = 1) -> 'QualityGate':
        """Validate DataFrame is not empty"""
        count = self.df.count()
        passed = count >= min_rows
        self.passed = self.passed and passed
        
        self.results.append({
            'check': 'Not Empty',
            'passed': passed,
            'details': f"Row count: {count} (minimum: {min_rows})",
            'severity': 'CRITICAL'
        })
        return self
    
    def check_no_nulls(self, columns: list) -> 'QualityGate':
        """Validate specified columns have no null values"""
        for col in columns:
            if col not in self.df.columns:
                self.results.append({
                    'check': f'No Nulls - {col}',
                    'passed': False,
                    'details': f"Column '{col}' not found in DataFrame",
                    'severity': 'HIGH'
                })
                self.passed = False
                continue
            
            null_count = self.df.filter(F.col(col).isNull()).count()
            total_count = self.df.count()
            passed = null_count == 0
            self.passed = self.passed and passed
            
            self.results.append({
                'check': f'No Nulls - {col}',
                'passed': passed,
                'details': f"Null count: {null_count}/{total_count} ({null_count/total_count*100:.2f}%)",
                'severity': 'HIGH'
            })
        return self
    
    def check_unique(self, columns: list) -> 'QualityGate':
        """Validate uniqueness of specified columns"""
        total_count = self.df.count()
        distinct_count = self.df.select(columns).distinct().count()
        duplicate_count = total_count - distinct_count
        
        passed = duplicate_count == 0
        self.passed = self.passed and passed
        
        self.results.append({
            'check': f'Uniqueness - {columns}',
            'passed': passed,
            'details': f"Duplicates: {duplicate_count}/{total_count} ({duplicate_count/total_count*100:.2f}%)",
            'severity': 'HIGH'
        })
        return self
    
    def check_value_range(self, column: str, min_val=None, max_val=None) -> 'QualityGate':
        """Validate values are within specified range"""
        if column not in self.df.columns:
            self.results.append({
                'check': f'Value Range - {column}',
                'passed': False,
                'details': f"Column '{column}' not found",
                'severity': 'MEDIUM'
            })
            self.passed = False
            return self
        
        condition = F.lit(True)
        if min_val is not None:
            condition = condition & (F.col(column) >= min_val)
        if max_val is not None:
            condition = condition & (F.col(column) <= max_val)
        
        invalid_count = self.df.filter(~condition).count()
        total_count = self.df.count()
        passed = invalid_count == 0
        self.passed = self.passed and passed
        
        range_str = f"[{min_val if min_val is not None else '-∞'}, {max_val if max_val is not None else '∞'}]"
        self.results.append({
            'check': f'Value Range - {column}',
            'passed': passed,
            'details': f"Out of range {range_str}: {invalid_count}/{total_count} ({invalid_count/total_count*100:.2f}%)",
            'severity': 'MEDIUM'
        })
        return self
    
    def check_freshness(self, timestamp_column: str, max_age_hours: int = 24) -> 'QualityGate':
        """Validate data freshness"""
        if timestamp_column not in self.df.columns:
            self.results.append({
                'check': 'Data Freshness',
                'passed': False,
                'details': f"Column '{timestamp_column}' not found",
                'severity': 'MEDIUM'
            })
            self.passed = False
            return self
        
        max_timestamp = self.df.agg(F.max(timestamp_column)).collect()[0][0]
        
        if max_timestamp is None:
            passed = False
            age_hours = None
        else:
            age = datetime.now() - max_timestamp
            age_hours = age.total_seconds() / 3600
            passed = age_hours <= max_age_hours
        
        self.passed = self.passed and passed
        
        self.results.append({
            'check': 'Data Freshness',
            'passed': passed,
            'details': f"Latest record: {max_timestamp}, Age: {age_hours:.1f}h (max: {max_age_hours}h)" if age_hours else "No timestamp data",
            'severity': 'MEDIUM'
        })
        return self
    
    def check_referential_integrity(self, column: str, reference_df: DataFrame, reference_column: str) -> 'QualityGate':
        """Validate foreign key relationships"""
        valid_values = reference_df.select(reference_column).distinct()
        
        invalid_count = self.df.join(
            valid_values,
            self.df[column] == valid_values[reference_column],
            "left_anti"
        ).count()
        
        total_count = self.df.count()
        passed = invalid_count == 0
        self.passed = self.passed and passed
        
        self.results.append({
            'check': f'Referential Integrity - {column}',
            'passed': passed,
            'details': f"Invalid references: {invalid_count}/{total_count} ({invalid_count/total_count*100:.2f}%)",
            'severity': 'HIGH'
        })
        return self
    
    def execute(self) -> dict:
        """Execute all checks and return results"""
        print("\n" + "="*80)
        print(f"QUALITY GATE REPORT: {self.table_name} ({self.layer.upper()} Layer)")
        print("="*80)
        print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total Checks: {len(self.results)}")
        
        passed_count = sum(1 for r in self.results if r['passed'])
        failed_count = len(self.results) - passed_count
        
        print(f"Passed: {passed_count} | Failed: {failed_count}")
        print("="*80)
        
        for result in self.results:
            status = "✓ PASS" if result['passed'] else "✗ FAIL"
            severity = result['severity']
            print(f"{status} [{severity:8}] {result['check']}")
            print(f"           {result['details']}")
        
        print("="*80)
        
        if self.passed:
            print("\n✓ ALL QUALITY GATES PASSED - Data approved for promotion")
        else:
            print("\n✗ QUALITY GATES FAILED - Data requires remediation")
        
        print("="*80 + "\n")
        
        return {
            'table_name': self.table_name,
            'layer': self.layer,
            'timestamp': datetime.now().isoformat(),
            'passed': self.passed,
            'total_checks': len(self.results),
            'passed_checks': passed_count,
            'failed_checks': failed_count,
            'results': self.results
        }

print("✓ Quality Gate framework loaded")

## 5. Standards Enforcement & Compliance

### Automated Enforcement Mechanisms

**1. Pre-Commit Validation**
* Schema validation before table creation
* Naming convention checks
* Required metadata validation

**2. Continuous Monitoring**
* Daily quality gate execution
* Automated alerts on failures
* Trend analysis and reporting

**3. Access Auditing**
* Regular permission reviews
* Access log analysis
* Anomaly detection

### Priority Dataset Classification

**Tier 1 - Critical:**
* Patient demographics
* Clinical encounters
* Medication orders
* Lab results
* **SLA:** 99.9% quality score, < 1 hour data latency

**Tier 2 - Important:**
* Operational metrics
* Financial transactions
* Inventory data
* **SLA:** 99% quality score, < 4 hour data latency

**Tier 3 - Standard:**
* Reference data
* Lookup tables
* Historical archives
* **SLA:** 95% quality score, < 24 hour data latency

In [0]:
# Automated standards enforcement

import re

class StandardsValidator:
    """Validate compliance with data governance standards"""
    
    def __init__(self):
        self.violations = []
    
    def validate_table_name(self, table_name: str) -> bool:
        """Validate table naming convention"""
        # Pattern: {domain}_{entity}_{type}
        pattern = r'^[a-z]+_[a-z_]+_(dim|fact|raw|agg|bridge|ref)$'
        
        if not re.match(pattern, table_name):
            self.violations.append({
                'rule': 'Table Naming Convention',
                'severity': 'HIGH',
                'message': f"Table name '{table_name}' does not follow pattern: {{domain}}_{{entity}}_{{type}}",
                'example': 'patient_demographics_dim, clinical_encounters_fact'
            })
            return False
        return True
    
    def validate_column_names(self, columns: list) -> bool:
        """Validate column naming conventions"""
        valid = True
        
        for col in columns:
            # Check snake_case
            if not re.match(r'^[a-z][a-z0-9_]*$', col):
                self.violations.append({
                    'rule': 'Column Naming Convention',
                    'severity': 'MEDIUM',
                    'message': f"Column '{col}' must use snake_case (lowercase with underscores)",
                    'example': 'patient_id, first_name, created_timestamp'
                })
                valid = False
            
            # Check for reserved suffixes
            if col.endswith('_id') and not col.replace('_id', '').isidentifier():
                self.violations.append({
                    'rule': 'ID Column Standard',
                    'severity': 'LOW',
                    'message': f"ID column '{col}' should follow pattern: {{entity}}_id",
                    'example': 'patient_id, encounter_id'
                })
                valid = False
        
        return valid
    
    def validate_table_properties(self, properties: dict) -> bool:
        """Validate required table properties"""
        required = [
            'owner', 'domain', 'pii_flag', 'retention_days',
            'data_classification', 'quality_tier', 'update_frequency',
            'source_system', 'business_owner'
        ]
        
        missing = [prop for prop in required if prop not in properties]
        
        if missing:
            self.violations.append({
                'rule': 'Required Table Properties',
                'severity': 'CRITICAL',
                'message': f"Missing required properties: {', '.join(missing)}",
                'example': 'All tables must have owner, domain, pii_flag, etc.'
            })
            return False
        
        # Validate property values
        valid_domains = ['clinical', 'operational', 'analytics', 'reference']
        if properties.get('domain') not in valid_domains:
            self.violations.append({
                'rule': 'Domain Classification',
                'severity': 'HIGH',
                'message': f"Invalid domain: {properties.get('domain')}. Must be one of: {valid_domains}",
                'example': 'domain=clinical'
            })
            return False
        
        valid_classifications = ['public', 'internal', 'confidential', 'restricted']
        if properties.get('data_classification') not in valid_classifications:
            self.violations.append({
                'rule': 'Data Classification',
                'severity': 'CRITICAL',
                'message': f"Invalid classification: {properties.get('data_classification')}. Must be one of: {valid_classifications}",
                'example': 'data_classification=confidential'
            })
            return False
        
        return True
    
    def validate_data_types(self, schema: StructType) -> bool:
        """Validate data type standards"""
        valid = True
        
        for field in schema.fields:
            # Check ID columns use appropriate types
            if field.name.endswith('_id'):
                if not isinstance(field.dataType, (IntegerType, StringType)):
                    self.violations.append({
                        'rule': 'ID Column Data Type',
                        'severity': 'MEDIUM',
                        'message': f"Column '{field.name}' should use BIGINT or STRING type",
                        'example': 'patient_id BIGINT'
                    })
                    valid = False
            
            # Check date columns use DATE type
            if field.name.endswith('_date'):
                if not isinstance(field.dataType, DateType):
                    self.violations.append({
                        'rule': 'Date Column Data Type',
                        'severity': 'MEDIUM',
                        'message': f"Column '{field.name}' should use DATE type",
                        'example': 'admission_date DATE'
                    })
                    valid = False
            
            # Check timestamp columns use TIMESTAMP type
            if field.name.endswith('_timestamp'):
                if not isinstance(field.dataType, TimestampType):
                    self.violations.append({
                        'rule': 'Timestamp Column Data Type',
                        'severity': 'MEDIUM',
                        'message': f"Column '{field.name}' should use TIMESTAMP type",
                        'example': 'created_timestamp TIMESTAMP'
                    })
                    valid = False
            
            # Check flag columns use BOOLEAN type
            if field.name.startswith('is_') or field.name.endswith('_flag'):
                if not isinstance(field.dataType, BooleanType):
                    self.violations.append({
                        'rule': 'Boolean Column Data Type',
                        'severity': 'LOW',
                        'message': f"Column '{field.name}' should use BOOLEAN type",
                        'example': 'is_active BOOLEAN'
                    })
                    valid = False
        
        return valid
    
    def report(self) -> bool:
        """Generate compliance report"""
        print("\n" + "="*80)
        print("STANDARDS COMPLIANCE REPORT")
        print("="*80)
        
        if not self.violations:
            print("\n✓ ALL STANDARDS CHECKS PASSED")
            print("  No violations detected")
            print("="*80 + "\n")
            return True
        
        print(f"\n✗ FOUND {len(self.violations)} VIOLATION(S)\n")
        
        # Group by severity
        critical = [v for v in self.violations if v['severity'] == 'CRITICAL']
        high = [v for v in self.violations if v['severity'] == 'HIGH']
        medium = [v for v in self.violations if v['severity'] == 'MEDIUM']
        low = [v for v in self.violations if v['severity'] == 'LOW']
        
        for severity, violations in [('CRITICAL', critical), ('HIGH', high), ('MEDIUM', medium), ('LOW', low)]:
            if violations:
                print(f"\n{severity} SEVERITY ({len(violations)}):")
                print("-" * 80)
                for v in violations:
                    print(f"\n  Rule: {v['rule']}")
                    print(f"  Issue: {v['message']}")
                    print(f"  Example: {v['example']}")
        
        print("\n" + "="*80)
        print("\n✗ STANDARDS VALIDATION FAILED - Remediation required")
        print("="*80 + "\n")
        
        return False

print("✓ Standards Validator loaded")

In [0]:
# Example: Validate standards compliance

validator = StandardsValidator()

# Validate table name
validator.validate_table_name("patient_demographics_dim")

# Validate column names
validator.validate_column_names(sample_df.columns)

# Validate table properties
properties = {
    'owner': 'data_engineering_team',
    'domain': 'clinical',
    'pii_flag': 'true',
    'retention_days': '2555',
    'data_classification': 'confidential',
    'quality_tier': 'silver',
    'update_frequency': 'daily',
    'source_system': 'emr_system',
    'business_owner': 'clinical.lead@example.com'
}
validator.validate_table_properties(properties)

# Validate data types
validator.validate_data_types(patient_schema)

# Generate compliance report
compliant = validator.report()

if compliant:
    print("✓ Table meets all governance standards")
else:
    print("✗ Table requires updates to meet standards")

## 6. Continuous Monitoring & Compliance

### Automated Monitoring Framework

**Daily Checks:**
* Quality gate execution on all Tier 1 datasets
* Schema drift detection
* Data volume anomaly detection
* Access pattern analysis

**Weekly Reviews:**
* Permission audit reports
* Data lineage validation
* Metadata completeness checks
* SLA compliance reporting

**Monthly Governance Reviews:**
* Catalog-wide compliance assessment
* Unused table identification
* Cost optimization opportunities
* Policy effectiveness evaluation

### Compliance Metrics

| Metric | Target | Measurement |
|--------|--------|-------------|
| **Metadata Completeness** | 100% | Tables with all required properties |
| **Quality Gate Pass Rate** | >95% | Successful validations / Total runs |
| **Access Compliance** | 100% | Proper RBAC implementation |
| **Data Freshness** | >98% | Tables meeting SLA thresholds |
| **Schema Standardization** | >90% | Tables following naming conventions |

#### **Auditing Queries**

```sql
-- Audit Query 1: Find tables without required metadata
SELECT 
  table_catalog,
  table_schema,
  table_name,
  table_type,
  comment
FROM prod_catalog.information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND (comment IS NULL OR comment = '')
ORDER BY table_schema, table_name;

-- Audit Query 2: Identify tables with missing ownership
SELECT 
  table_catalog,
  table_schema,
  table_name,
  table_owner
FROM prod_catalog.information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND (table_owner IS NULL OR table_owner = '')
ORDER BY table_schema, table_name;

-- Audit Query 3: Check for non-standard table names
SELECT 
  table_catalog,
  table_schema,
  table_name,
  CASE 
    WHEN table_name NOT REGEXP '^[a-z]+_[a-z_]+_(dim|fact|raw|agg|bridge|ref)$' 
    THEN 'Non-standard naming'
    ELSE 'Compliant'
  END AS naming_status
FROM prod_catalog.information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND table_name NOT REGEXP '^[a-z]+_[a-z_]+_(dim|fact|raw|agg|bridge|ref)$'
ORDER BY table_schema, table_name;

-- Audit Query 4: Find tables not accessed in last 30 days
SELECT 
  table_catalog,
  table_schema,
  table_name,
  DATEDIFF(CURRENT_DATE(), last_altered) AS days_since_modified
FROM prod_catalog.information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND DATEDIFF(CURRENT_DATE(), last_altered) > 30
ORDER BY days_since_modified DESC;

-- Audit Query 5: Review current permissions by principal
SHOW GRANTS ON CATALOG prod_catalog;
```

#### **Collect governance metrics for dashboard reporting**

In [0]:
from pyspark.sql import functions as F
from datetime import datetime, timedelta

class GovernanceMetrics:
    """Collect and report governance metrics"""
    
    def __init__(self, catalog: str):
        self.catalog = catalog
        self.metrics = {}
    
    def get_table_count_by_layer(self):
        """Count tables in each layer"""
        query = f"""
        SELECT 
            table_schema AS layer,
            COUNT(*) AS table_count
        FROM {self.catalog}.information_schema.tables
        WHERE table_schema IN ('bronze', 'silver', 'gold', 'sandbox')
        GROUP BY table_schema
        ORDER BY table_schema
        """
        
        df = spark.sql(query)
        self.metrics['table_count_by_layer'] = df.toPandas().to_dict('records')
        return df
    
    def get_metadata_completeness(self):
        """Calculate metadata completeness percentage"""
        query = f"""
        SELECT 
            table_schema AS layer,
            COUNT(*) AS total_tables,
            SUM(CASE WHEN comment IS NOT NULL AND comment != '' THEN 1 ELSE 0 END) AS tables_with_comments,
            ROUND(SUM(CASE WHEN comment IS NOT NULL AND comment != '' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS completeness_pct
        FROM {self.catalog}.information_schema.tables
        WHERE table_schema IN ('bronze', 'silver', 'gold')
        GROUP BY table_schema
        ORDER BY table_schema
        """
        
        df = spark.sql(query)
        self.metrics['metadata_completeness'] = df.toPandas().to_dict('records')
        return df
    
    def get_column_count_stats(self):
        """Get column count statistics by layer"""
        query = f"""
        SELECT 
            table_schema AS layer,
            COUNT(DISTINCT table_name) AS table_count,
            COUNT(*) AS total_columns,
            ROUND(AVG(column_count), 2) AS avg_columns_per_table
        FROM (
            SELECT 
                table_schema,
                table_name,
                COUNT(*) AS column_count
            FROM {self.catalog}.information_schema.columns
            WHERE table_schema IN ('bronze', 'silver', 'gold')
            GROUP BY table_schema, table_name
        )
        GROUP BY table_schema
        ORDER BY table_schema
        """
        
        df = spark.sql(query)
        self.metrics['column_stats'] = df.toPandas().to_dict('records')
        return df
    
    def get_data_classification_summary(self):
        """Summarize tables by data classification (from table properties)"""
        # Note: This would query table properties if stored
        # For now, return a placeholder structure
        classification_data = [
            {'classification': 'confidential', 'table_count': 0, 'percentage': 0.0},
            {'classification': 'internal', 'table_count': 0, 'percentage': 0.0},
            {'classification': 'public', 'table_count': 0, 'percentage': 0.0}
        ]
        
        self.metrics['data_classification'] = classification_data
        print("Note: Data classification metrics require table properties to be set")
        return spark.createDataFrame(classification_data)
    
    def generate_report(self):
        """Generate comprehensive governance report"""
        print("\n" + "="*80)
        print(f"GOVERNANCE METRICS REPORT: {self.catalog}")
        print("="*80)
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80)
        
        print("\n1. TABLE COUNT BY LAYER")
        print("-" * 80)
        df1 = self.get_table_count_by_layer()
        display(df1)
        
        print("\n2. METADATA COMPLETENESS")
        print("-" * 80)
        df2 = self.get_metadata_completeness()
        display(df2)
        
        print("\n3. COLUMN STATISTICS")
        print("-" * 80)
        df3 = self.get_column_count_stats()
        display(df3)
        
        print("\n4. DATA CLASSIFICATION SUMMARY")
        print("-" * 80)
        df4 = self.get_data_classification_summary()
        display(df4)
        
        print("\n" + "="*80)
        print("END OF REPORT")
        print("="*80 + "\n")
        
        return self.metrics

# Example usage
print("Governance Metrics collector initialized")
print("\nUsage: metrics = GovernanceMetrics('prod_catalog')")
print("       metrics.generate_report()")